In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
%%time
basedir = './공시지가/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]
land_prices_df = pd.read_csv(basedir + filenames[0], encoding='euc-kr')

<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 21.2 s


In [3]:
print(land_prices_df.shape)
land_prices_df.head()

(29393777, 12)


,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준년도,기준월,공시지가,공시일자,표준지여부,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1990,1,750000,1990-08-30,N,2020-09-26
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1991,1,980000,1991-06-29,N,2020-09-26
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1992,1,980000,1992-06-01,N,2020-09-26
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1993,1,980000,1993-05-22,N,2020-09-26
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1994,1,960000,1994-06-30,N,2020-09-26


In [4]:
land_prices_df = land_prices_df[land_prices_df['기준년도'] == 2020]

In [5]:
land_prices_df.shape

(906224, 12)

In [6]:
def landnum_modifier(x):
    # x is a string
    
    splitted = x.split('-')
    if len(splitted) == 1:
        return x+'-0'
    elif len(splitted) == 2:
        return x
    else:
        raise ValueError('myerror')

In [7]:
land_prices_df['지번주소'] = land_prices_df['법정동명'] + ' ' + land_prices_df['지번'].apply(landnum_modifier)
land_prices_df.head()

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준년도,기준월,공시지가,공시일자,표준지여부,데이터기준일자,지번주소
30,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2020,1,4357000,2020-05-29,N,2020-09-26,서울특별시 종로구 청운동 1-0
42,1111010100100010001,1111010100,서울특별시 종로구 청운동,1,일반,1-1,2020,1,1392000,2020-05-29,N,2020-09-26,서울특별시 종로구 청운동 1-1
73,1111010100100010002,1111010100,서울특별시 종로구 청운동,1,일반,1-2,2020,1,2520000,2020-05-29,N,2020-09-26,서울특별시 종로구 청운동 1-2
104,1111010100100010003,1111010100,서울특별시 종로구 청운동,1,일반,1-3,2020,1,4337000,2020-05-29,N,2020-09-26,서울특별시 종로구 청운동 1-3
116,1111010100100010004,1111010100,서울특별시 종로구 청운동,1,일반,1-4,2020,1,1554000,2020-05-29,N,2020-09-26,서울특별시 종로구 청운동 1-4


In [8]:
land_prices_df = land_prices_df.sort_values(['지번주소', '기준년도', '기준월']).reset_index(drop=True)

In [9]:
land_prices_df.drop_duplicates(subset=['지번주소'], inplace=True)

In [10]:
land_prices_df.shape

(903493, 13)

In [11]:
land_prices_df['특수지구분명'].value_counts()

일반    890240
산      13253
Name: 특수지구분명, dtype: int64

In [12]:
land_prices_df = land_prices_df[['지번주소', '특수지구분명', '기준년도', '공시지가', '표준지여부']]

In [13]:
land_prices_df.head()

,지번주소,특수지구분명,기준년도,공시지가,표준지여부
0,서울특별시 강남구 개포동 100-0,일반,2020,396000,N
1,서울특별시 강남구 개포동 101-0,일반,2020,475200,N
2,서울특별시 강남구 개포동 102-0,일반,2020,475200,N
3,서울특별시 강남구 개포동 103-0,일반,2020,309700,N
4,서울특별시 강남구 개포동 104-1,일반,2020,394600,N


In [14]:
land_prices_df.to_csv('./공시지가/land_prices_production_ver.csv', index=False)